<a href="https://colab.research.google.com/github/Axel0087/NLP2023/blob/main/week39%2B41_Huggingface_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load af Dataset og grundfunktioner

In [3]:
%%capture
!pip install datasets
!pip install bpemb

In [4]:
from torch.utils.data import DataLoader, Dataset
import nltk
from bpemb import BPEmb
import string
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from torch import nn
import torch

In [5]:
dataset = load_dataset("copenlu/answerable_tydiqa")
nltk.download('punkt')

#train_set = dataset["train"]
#validation_set = dataset["validation"]

def get_answer_start(row):
  return row["annotations"]["answer_start"][0]

def get_answer(row):
  return row["annotations"]["answer_text"][0]

def get_document(row):
  return row["document_plaintext"]

def get_question(row):
  return row["question_text"]

def oracle(answer, document):
  return answer != "" and answer in document

def get_language(dataset, lang):
  return dataset.filter(lambda x: x["language"] == lang)#[row for row in dataset if row['language'] == lang]

def get_answerble(dataset):
  return dataset.filter(lambda x: x["annotations"]["answer_start"] != [-1])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def add_index(examples, idx):
    # Adding an 'id' field with the index value
    examples["id"] = idx
    return examples

In [7]:
dataset_idx = dataset.map(add_index, with_indices=True)

In [8]:
print(dataset_idx['train'][0])

{'question_text': 'Milloin Charles Fort syntyi?', 'document_title': 'Charles Fort', 'language': 'finnish', 'annotations': {'answer_start': [18], 'answer_text': ['6. elokuuta (joidenkin lähteiden mukaan 9.) 1874']}, 'document_plaintext': 'Charles Hoy Fort (6. elokuuta (joidenkin lähteiden mukaan 9.) 1874 – 3. toukokuuta 1932) oli yhdysvaltalainen kirjailija ja paranormaalien ilmiöiden tutkija.', 'document_url': 'https://fi.wikipedia.org/wiki/Charles%20Fort', 'id': 0}


In [9]:
train_set=dataset_idx['train']
validation_set=dataset_idx['validation']

### English

In [10]:
train_english = get_language(train_set, "english")
val_english = get_language(validation_set, "english")

In [11]:
train_english_answerble = get_answerble(train_english)
validation_english_answerble = get_answerble(val_english)

### language data

In [12]:
train_arabic = get_language(train_set, "arabic")
val_arabic = get_language(validation_set, "arabic")

train_bengali = get_language(train_set, "bengali")
val_bengali = get_language(validation_set, "bengali")

train_indonesian = get_language(train_set, "indonesian")
val_indonesian = get_language(validation_set, "indonesian")

In [13]:
train_arabic_answerble = get_answerble(train_arabic)
val_arabic_answerble = get_answerble(val_arabic)

train_bengali_answerble = get_answerble(train_bengali)
val_bengali_answerble = get_answerble(val_bengali)

train_indonesian_answerble = get_answerble(train_indonesian)
val_indonesian_answerble = get_answerble(val_indonesian)

#Roberta

In [14]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")


In [15]:
!pip install update transformers


In [16]:
from datasets import load_dataset
from datasets import load_metric
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import AutoConfig
from functools import partial
import torch
import random
import numpy as np
from tqdm import tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import LambdaLR
from torch import nn
from collections import defaultdict, OrderedDict
MODEL_NAME = 'xlm-roberta-base'
#MODEL_NAME = 'bert-base-uncased'

In [17]:
""" Official evaluation script for v1.1 of the SQuAD dataset. """
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate_squad(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

def compute_squad(predictions, references):
  pred_dict = {prediction["id"]: prediction["prediction_text"] for prediction in predictions}
  dataset = [
      {
          "paragraphs": [
              {
                  "qas": [
                      {
                          "answers": [{"text": answer_text} for answer_text in ref["answers"]["answer_text"]],
                          "id": ref["id"],
                      }
                      for ref in references
                  ]
              }
          ]
      }
  ]
  score = evaluate_squad(dataset=dataset, predictions=pred_dict)
  return score

In [39]:
tk = AutoTokenizer.from_pretrained(MODEL_NAME)


## train_features

In [38]:
def get_train_features(tk, samples):
  '''
  Tokenizes all of the text in the given samples, splittling inputs that are too long for our model
  across multiple features. Finds the token offsets of the answers, which serve as the labels for
  our inputs.
  '''
  batch = tk.batch_encode_plus(
        [[q,c] for q,c in zip(samples['question_text'], samples['document_plaintext'])],
        padding='max_length',
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )

  # Get a list which maps the input features index to their original index in the
  # samples list (for split inputs). E.g. if our batch size is 4 and the second sample
  # is split into 3 inputs because it is very large, sample_mapping would look like
  # [0, 1, 1, 1, 2, 3]
  sample_mapping = batch.pop('overflow_to_sample_mapping')
  # Get all of the character offsets for each token
  offset_mapping = batch.pop('offset_mapping')

  # Store the start and end tokens
  batch['start_tokens'] = []
  batch['end_tokens'] = []

  # Iterate through all of the offsets
  for i, offsets in enumerate(offset_mapping):
    # Get the right sample by mapping it to its original index
    sample_idx = sample_mapping[i]
    # Get the sequence IDs to know where context starts so we can ignore question tokens
    sequence_ids = batch.sequence_ids(i)

    # Get the start and end character positions of the answer
    ans = samples['annotations'][sample_idx]
    start_char = ans['answer_start'][0]
    end_char = start_char + len(ans['answer_text'][0])
    # while end_char > 0 and (end_char >= len(samples['context'][sample_idx]) or samples['context'][sample_idx][end_char] == ' '):
    #   end_char -= 1

    # Start from the first token in the context, which can be found by going to the
    # first token where sequence_ids is 1
    start_token = 0
    while sequence_ids[start_token] != 1:
      start_token += 1

    end_token = len(offsets) - 1
    while sequence_ids[end_token] != 1:
      end_token -= 1

    # By default set it to the CLS token if the answer isn't in this input
    if start_char < offsets[start_token][0] or end_char > offsets[end_token][1]:
      start_token = 0
      end_token = 0
    # Otherwise find the correct token indices
    else:
      # Advance the start token index until we have passed the start character index
      while start_token < len(offsets) and offsets[start_token][0] <= start_char:
        start_token += 1
      start_token -= 1

      # Decrease the end token index until we have passed the end character index
      while end_token >= 0 and offsets[end_token][1] >= end_char:
        end_token -= 1
      end_token += 1

    batch['start_tokens'].append(start_token)
    batch['end_tokens'].append(end_token)

  #batch['start_tokens'] = np.array(batch['start_tokens'])
  #batch['end_tokens'] = np.array(batch['end_tokens'])

  return batch

def collate_fn(inputs):
  '''
  Defines how to combine different samples in a batch
  '''
  input_ids = torch.tensor([i['input_ids'] for i in inputs])
  attention_mask = torch.tensor([i['attention_mask'] for i in inputs])
  start_tokens = torch.tensor([i['start_tokens'] for i in inputs])
  end_tokens = torch.tensor([i['end_tokens'] for i in inputs])

  # Truncate to max length
  max_len = max(attention_mask.sum(-1))
  input_ids = input_ids[:,:max_len]
  attention_mask = attention_mask[:,:max_len]

  return {'input_ids': input_ids, 'attention_mask': attention_mask, 'start_tokens': start_tokens, 'end_tokens': end_tokens}

### example with english

In [18]:
tokenized_dataset= train_english_answerble.map(partial(get_train_features, tk), batched=True, remove_columns=train_english_answerble.column_names)

Map:   0%|          | 0/3696 [00:00<?, ? examples/s]

In [19]:
train_english_answerble[0]

{'question_text': 'When was quantum field theory developed?',
 'document_title': 'Quantum field theory',
 'language': 'english',
 'annotations': {'answer_start': [159], 'answer_text': ['1920s']},
 'document_plaintext': 'Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1',
 'document_url': 'https://en.wikipedia.org/wiki/Quantum%20field%20theory',
 'id': 26}

In [20]:
tokenized_dataset[0]

{'input_ids': [0,
  14847,
  509,
  110436,
  44457,
  154453,
  126809,
  32,
  2,
  2,
  75344,
  316,
  44457,
  154453,
  6083,
  538,
  80723,
  678,
  70,
  35187,
  111,
  77556,
  155116,
  1771,
  182809,
  7,
  4,
  237,
  70,
  77556,
  155116,
  1771,
  44457,
  509,
  70,
  4734,
  51529,
  54704,
  289,
  44457,
  237,
  111,
  70,
  23031,
  7,
  5,
  75064,
  19721,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [21]:
tk.decode(tokenized_dataset['input_ids'][0])


'<s> When was quantum field theory developed?</s></s> Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

## train

In [40]:
def train(
    model: nn.Module,
    train_dl: DataLoader,
    optimizer: torch.optim.Optimizer,
    schedule: LambdaLR,
    n_epochs: int,
    device: torch.device
):
  """
  The main training loop which will optimize a given model on a given dataset
  :param model: The model being optimized
  :param train_dl: The training dataset
  :param optimizer: The optimizer used to update the model parameters
  :param n_epochs: Number of epochs to train for
  :param device: The device to train on
  """

  # Keep track of the loss and best accuracy
  losses = []
  best_acc = 0.0
  pcounter = 0

  # Iterate through epochs
  for ep in range(n_epochs):

    loss_epoch = []

    #Iterate through each batch in the dataloader
    for batch in tqdm(train_dl):
      # VERY IMPORTANT: Make sure the model is in training mode, which turns on
      # things like dropout and layer normalization
      model.train()

      # VERY IMPORTANT: zero out all of the gradients on each iteration -- PyTorch
      # keeps track of these dynamically in its computation graph so you need to explicitly
      # zero them out
      optimizer.zero_grad()

      # Place each tensor on the GPU
      batch = {b: batch[b].to(device) for b in batch}

      # Pass the inputs through the model, get the current loss and logits
      outputs = model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask'],
          start_positions=batch['start_tokens'],
          end_positions=batch['end_tokens']
      )
      loss = outputs['loss']
      losses.append(loss.item())
      loss_epoch.append(loss.item())

      # Calculate all of the gradients and weight updates for the model
      loss.backward()

      # Optional: clip gradients
      #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Finally, update the weights of the model and advance the LR schedule
      optimizer.step()
      scheduler.step()
      #gc.collect()
  return losses

# validate and predict

In [22]:
def get_validation_features(tk, samples):
  # First, tokenize the text. We get the offsets and return overflowing sequences in
  # order to break up long sequences into multiple inputs. The offsets will help us
  # determine the original answer text
  batch = tk.batch_encode_plus(
        [[q,c] for q,c in zip(samples['question_text'], samples['document_plaintext'])],
        padding='max_length',
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )

  # We'll store the ID of the samples to calculate squad score
  batch['example_id'] = []
  # The overflow sample map tells us which input each sample corresponds to
  sample_map = batch.pop('overflow_to_sample_mapping')

  for i in range(len(batch['input_ids'])):
    # The sample index tells us which of the values in "samples" these features belong to
    sample_idx = sample_map[i]
    sequence_ids = batch.sequence_ids(i)

    # Add the ID to map these features back to the correct sample
    batch['example_id'].append(samples['id'][sample_idx])

    #Set offsets for non-context words to be None for ease of processing
    batch['offset_mapping'][i] = [o if sequence_ids[k] == 1 else None for k,o in enumerate(batch['offset_mapping'][i])]

  return batch

def val_collate_fn(inputs):
  input_ids = torch.tensor([i['input_ids'] for i in inputs])
  attention_mask = torch.tensor([i['attention_mask'] for i in inputs])

  # Truncate to max length
  max_len = max(attention_mask.sum(-1))
  input_ids = input_ids[:,:max_len]
  attention_mask = attention_mask[:,:max_len]

  return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [23]:
def predict(model: nn.Module, valid_dl: DataLoader):
  """
  Evaluates the model on the given dataset
  :param model: The model under evaluation
  :param valid_dl: A `DataLoader` reading validation data
  :return: The accuracy of the model on the dataset
  """
  # VERY IMPORTANT: Put your model in "eval" mode -- this disables things like
  # layer normalization and dropout
  model.eval()
  start_logits_all = []
  end_logits_all = []

  # ALSO IMPORTANT: Don't accumulate gradients during this process
  with torch.no_grad():
    for batch in tqdm(valid_dl, desc='Evaluation'):
      batch = {b: batch[b].to(device) for b in batch}

      # Pass the inputs through the model, get the current loss and logits
      outputs = model(
          input_ids=batch['input_ids'],
          attention_mask=batch['attention_mask']
      )
      # Store the "start" class logits and "end" class logits for every token in the input
      start_logits_all.extend(list(outputs['start_logits'].detach().cpu().numpy()))
      end_logits_all.extend(list(outputs['end_logits'].detach().cpu().numpy()))


    return start_logits_all,end_logits_all

def post_process_predictions(examples, dataset, logits, num_possible_answers = 20, max_answer_length = 30):
  all_start_logits, all_end_logits = logits
  # Build a map from example to its corresponding features. This will allow us to index from
  # sample ID to all of the features for that sample (in case they were split up due to long input)
  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = defaultdict(list)
  for i, feature in enumerate(dataset):
      features_per_example[example_id_to_index[feature["example_id"]]].append(i)

  # Create somewhere to store our predictions
  predictions = OrderedDict()

  # Iterate through each sample in the dataset
  for j, sample in enumerate(tqdm(examples)):

    # Get the feature indices (all of the features split across the batch)
    feature_indices = features_per_example[j]
    # Get the original context which predumably has the answer text
    context = sample['document_plaintext']

    preds = []
    # Iterate through all of the features
    for ft_idx in feature_indices:

      # Get the start and end answer logits for this input
      start_logits = all_start_logits[ft_idx]
      end_logits = all_end_logits[ft_idx]

      # Get the offsets to map token indices to character indices
      offset_mapping = dataset[ft_idx]['offset_mapping']

      # Sort the logits and take the top N
      start_indices = np.argsort(start_logits)[::-1][:num_possible_answers]
      end_indices = np.argsort(end_logits)[::-1][:num_possible_answers]

      # Iterate through start and end indices
      for start_index in start_indices:
        for end_index in end_indices:

          # Ignore this combination if either the indices are not in the context
          if start_index >= len(offset_mapping) or end_index >= len(offset_mapping) or offset_mapping[start_index] is None or offset_mapping[end_index] is None:
            continue

          # Also ignore if the start index is greater than the end index of the number of tokens
          # is greater than some specified threshold
          if start_index > end_index or end_index - start_index + 1 > max_answer_length:
            continue

          ans_text = context[offset_mapping[start_index][0]:offset_mapping[end_index][1]]
          preds.append({
              'score': start_logits[start_index] + end_logits[end_index],
              'text': ans_text
          })

    if len(preds) > 0:
      # Sort by score to get the top answer
      answer = sorted(preds, key=lambda x: x['score'], reverse=True)[0]
    else:
      answer = {'score': 0.0, 'text': ""}

    predictions[sample['id']] = answer['text']
  return predictions

# train model

In [37]:
# Create the optimizer
lr=2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200

tk = AutoTokenizer.from_pretrained(MODEL_NAME)


In [41]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
runs = [#("English", train_english_answerble),
        # ("Arabic", train_arabic_answerble,'simoneZethof/Arabic_multi'),
         ("Bengali", train_bengali_answerble,'simoneZethof/Bengali_multi'),
       #  ("Indonesian", train_indonesian_answerble,'simoneZethof/Indonesian_multi')
]

In [43]:
result_model = {}

In [44]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

print(device)
for (tag, ds_train,hugging) in runs:

  model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)

  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

  print(tag)
  tokenized_dataset = ds_train.map(partial(get_train_features, tk), batched=True, remove_columns=ds_train.column_names)
  samples = random.sample(list(range(len(tokenized_dataset))), len(tokenized_dataset))
  tokenized_dataset = tokenized_dataset.select(samples)
  train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)

  scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps,n_epochs * len(train_dl))
  losses = train(
    model,
    train_dl,
    optimizer,
    scheduler,
    n_epochs,
    device)

  result_model[tag] = [losses, model]
  tk.push_to_hub(hugging)
  model.push_to_hub(hugging)

cuda


Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bengali


 50%|████▉     | 319/639 [01:54<01:54,  2.79it/s]


KeyboardInterrupt: ignored

# zero-shot cross-lingual results for sequence labeller

In [24]:
pip install torchmetrics

In [25]:
from torchmetrics.text import BLEUScore

from transformers import pipeline



In [ ]:
result_crossed = {}

In [30]:
runs_val = [#("English", train_english_answerble, validation_english_answerble),
          #("Arabic", val_arabic_answerble),
          ("Bengali", val_bengali_answerble),
          ("Indonesian", val_indonesian_answerble)]

for (trained_model_tag, _, hugging) in runs:
  tokenizer = AutoTokenizer.from_pretrained(hugging)
  model_trained =AutoModelForQuestionAnswering.from_pretrained(hugging).to(device)

  for (tag, ds_val ) in runs_val:
    print(device)
    validation_dataset = ds_val.map(partial(get_validation_features, tokenizer), batched=True, remove_columns=ds_val.column_names)
    val_dl = DataLoader(validation_dataset, collate_fn=val_collate_fn, batch_size=32)
    logits = predict(model_trained, val_dl)
    predictions = post_process_predictions(ds_val, validation_dataset, logits)
    formatted_predictions = [{'id': k, 'prediction_text': v} for k,v in predictions.items()]
    gold = [{'id': example['id'], 'answers': example['annotations']} for example in ds_val]
    result = compute_squad(references=gold, predictions=formatted_predictions)

    bleu = BLEUScore(n_gram=1)
    preds = []
    target = []

    for row in range(len(formatted_predictions)):
      preds.append(formatted_predictions[row]['prediction_text'])
      target.append(gold[row]['answers']['answer_text'])
  #  print(preds)
   # print(target)
    result['BLEU'] =bleu(preds, target)
    result_crossed[trained_model_tag,tag] = result
    print(f'\n Model trained on Language : {trained_model_tag} \n Validated on Language : {tag} \n {result} \n \n ')


cuda


100%|██████████| 112/112 [00:00<00:00, 507.60it/s]



 Model trained on Language : Arabic 
 Validated on Language : Bengali 
 {'exact_match': 40.17857142857143, 'f1': 53.128538128538125, 'BLEU': tensor(0.4624)} 
 
 
cuda


100%|██████████| 597/597 [00:01<00:00, 589.58it/s]



 Model trained on Language : Arabic 
 Validated on Language : Indonesian 
 {'exact_match': 59.79899497487437, 'f1': 74.59017654308826, 'BLEU': tensor(0.6713)} 
 
 
cuda


...

100%|██████████| 112/112 [00:00<00:00, 523.86it/s]


 Model trained on Language : Indonesian 
 Validated on Language : Bengali 
 {'exact_match': 36.607142857142854, 'f1': 50.45318768533053, 'BLEU': tensor(0.4533)} 
 
 
cuda


Map:   0%|          | 0/597 [00:00<?, ? examples/s]

100%|██████████| 597/597 [00:01<00:00, 570.82it/s]



 Model trained on Language : Indonesian 
 Validated on Language : Indonesian 
 {'exact_match': 68.00670016750419, 'f1': 79.36000054096237, 'BLEU': tensor(0.7344)} 
 
 


## eksempler

In [91]:
id_30 = validation_english_answerble.filter(lambda x: x["id"]== 30)
id_30[0]

{'question_text': 'What is a way to increase your wound healing speed?',
 'document_title': 'Wound healing',
 'language': 'english',
 'annotations': {'answer_start': [51],
  'answer_text': ['cleaning and protection from reinjury or infection']},
 'document_plaintext': "Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care.",
 'document_url': 'https://en.wikipedia.org/wiki/Wound%20healing',
 'id': 30}

In [111]:
print(formatted_predictions[300:302])
print(gold[300:302])

[{'id': 3736, 'prediction_text': 'at least one primary external dimension of less than 100 nanometers'}, {'id': 3744, 'prediction_text': '14th century'}]
[{'id': 3736, 'answers': {'answer_start': [235], 'answer_text': ['nanometers']}}, {'id': 3744, 'answers': {'answer_start': [38], 'answer_text': ['14th century']}}]


In [112]:
compute_squad(references=gold[300:302], predictions=formatted_predictions[300:302])

{'exact_match': 50.0, 'f1': 58.333333333333336}

In [108]:
compute_squad(references=gold[300:301], predictions=formatted_predictions[300:301])

{'exact_match': 0.0, 'f1': 16.666666666666668}

In [110]:
compute_squad(references=gold[301:302], predictions=formatted_predictions[301:302])

{'exact_match': 100.0, 'f1': 100.0}